<br/>

<!-- <p style="text-align:center;font-weight:bold;font-size:400%">Model-Free Prediction - Part 2</p> -->
<img src="assets/title_part2.png"/>

# Introduction

Part II of algorithms presented in Lecture 4 of UCL RL course by David Silver. Part I is [here](ModelFreePrediction_Part1.ipynb).

Notes:
* As in Part I, we are doing fixed-policy evaluation, no policy improvement
* Note that only Backward View TD(λ) is applicable in real world. N-Step and Forward TD(λ) are extreemly slow.


Algorithms:
* N-Step Return
* TD(λ) - Forward View
* TD(λ) - Backward View (eligibility traces)

Sources:
* UCL Course on RL: http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching.html
  * Lecture 4 pdf: http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching_files/MC-TD.pdf
  * Lecture 4 vid: https://www.youtube.com/watch?v=PnHCvfgC_ZA

Imports:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Introduction

In part I we explored some basic MC and TD algorithms. Both MC and TD are just special cases of broader class of algorithms, which we will implement in this notebook.

Blackjack environment is all good, but most trajectories will be max 2-3 steps long. To better demonstrate N-Step and TD(λ) we will need environment that is still simple, but allows for longer trajectories. As per lectures we will use a 1d-corridor.

# 1D-Corridor

<img src="assets/corridor.png"/>